In [2]:
# Importamos las librerías
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.callbacks import ModelCheckpoint

2025-08-12 12:45:07.455931: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-12 12:45:07.477569: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-12 12:45:07.643669: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-12 12:45:07.739318: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755002707.831985    3572 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755002707.84

In [ ]:
# Tamaño de las imágenes 
size = 150

# Generador para los datos de entrenamiento
train_datagen = ImageDataGenerator(rescale=1./255)

# Generador para los datos de validación
val_datagen = ImageDataGenerator(rescale=1./255)

# Cargamos las imágenes
train_generator = train_datagen.flow_from_directory(
        'data/train',
        target_size=(size, size),
        batch_size=20,
        class_mode='binary')

validation_generator = val_datagen.flow_from_directory(
        'data/test',
        target_size=(size, size),
        batch_size=20,
        class_mode='binary')

In [ ]:
modelo = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(size, size, 3)),
    MaxPooling2D(2, 2),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compilamos el modelo
modelo.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
# Guardamos el mejor modelo durante el entrenamiento
checkpoint = ModelCheckpoint('mejor_modelo.h5', 
                           monitor='val_accuracy',
                           save_best_only=True,
                           verbose=1)

# Entrenamiento
historia = modelo.fit(
      train_generator,
      steps_per_epoch=100,  # Número de lotes por época
      epochs=5,
      validation_data=validation_generator,
      validation_steps=50,  # Número de lotes para validación
      callbacks=[checkpoint],
      verbose=1)

In [ ]:
# Cargamos el mejor modelo guardado
from tensorflow.keras.models import load_model
modelo_final = load_model('mejor_modelo.h5')

# Función para probar una imagen
def predecir_imagen(ruta):
    img = plt.imread(ruta)
    plt.imshow(img)
    plt.axis('off')
    
    # Preprocesamiento
    img = tf.image.resize(img, [size, size])
    img = np.expand_dims(img, axis=0) / 255.0
    
    # Predicción
    prediccion = modelo_final.predict(img)
    if prediccion < 0.5:
        plt.title(f'Gato ({100*(1-prediccion[0][0]):.1f}%)')
    else:
        plt.title(f'Perro ({100*prediccion[0][0]:.1f}%)')
    plt.show()

# Probamos con una imagen
predecir_imagen('imagen.jpg')